## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,59.27,57,0,12.19,clear sky
1,1,Tadine,NC,-21.5500,167.8833,71.19,74,22,11.92,light rain
2,2,Busselton,AU,-33.6500,115.3333,59.09,88,100,17.05,light rain
3,3,Faanui,PF,-16.4833,-151.7500,81.79,73,5,13.60,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,75.54,91,94,10.00,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))


What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Faanui,PF,-16.4833,-151.7500,81.79,73,5,13.60,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,75.54,91,94,10.00,overcast clouds
7,7,Butaritari,KI,3.0707,172.7902,83.61,79,100,19.51,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,78.96,94,75,10.36,light rain
14,14,Port Augusta,AU,-32.5000,137.7667,77.70,27,96,2.17,overcast clouds
18,18,Mahebourg,MU,-20.4081,57.7000,76.24,64,40,13.80,scattered clouds
19,19,Rikitea,PF,-23.1203,-134.9692,76.37,61,100,9.48,overcast clouds
25,25,Atuona,PF,-9.8000,-139.0333,79.90,71,10,17.09,clear sky
26,26,Mana,GF,5.6592,-53.7767,76.55,91,100,9.35,overcast clouds
31,31,Bengkulu,ID,-3.8004,102.2655,81.82,69,83,4.59,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis = 0, how = 'any')

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Faanui,PF,81.79,clear sky,-16.4833,-151.7500,
4,Puerto Ayora,EC,75.54,overcast clouds,-0.7393,-90.3518,
7,Butaritari,KI,83.61,overcast clouds,3.0707,172.7902,
11,Vaini,TO,78.96,light rain,-21.2000,-175.2000,
14,Port Augusta,AU,77.70,overcast clouds,-32.5000,137.7667,
18,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,
19,Rikitea,PF,76.37,overcast clouds,-23.1203,-134.9692,
25,Atuona,PF,79.90,clear sky,-9.8000,-139.0333,
26,Mana,GF,76.55,overcast clouds,5.6592,-53.7767,
31,Bengkulu,ID,81.82,broken clouds,-3.8004,102.2655,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis = 0, how = 'any')
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Faanui,PF,81.79,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
4,Puerto Ayora,EC,75.54,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Butaritari,KI,83.61,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
11,Vaini,TO,78.96,light rain,-21.2000,-175.2000,Keleti Beach Resort
14,Port Augusta,AU,77.70,overcast clouds,-32.5000,137.7667,Majestic Oasis Apartments
18,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
19,Rikitea,PF,76.37,overcast clouds,-23.1203,-134.9692,Pension Maro'i
25,Atuona,PF,79.90,clear sky,-9.8000,-139.0333,Villa Enata
26,Mana,GF,76.55,overcast clouds,5.6592,-53.7767,LE SAMANA
31,Bengkulu,ID,81.82,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
 #            max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))